In [1]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 
from pathlib import Path


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 50

paths = Path(args.train_folder).glob('**/chorale_*.csv')
train_np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(train_np_list[0]))

paths = Path(args.val_folder).glob('**/chorale_*.csv')
val_np_list = [pd.read_csv(p).values.tolist() for p in paths]

paths = Path(args.test_folder).glob('**/chorale_*.csv')
test_np_list = [pd.read_csv(p).values.tolist() for p in paths]

print(train_np_list[0])

[[66, 61, 57, 54], [66, 61, 57, 54], [68, 61, 59, 54], [68, 61, 59, 54], [69, 66, 61, 54], [69, 66, 61, 56], [69, 66, 61, 57], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 59, 49], [68, 65, 59, 49], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 59, 50], [66, 66, 59, 50], [68, 66, 59, 50], [68, 66, 59, 50], [69, 66, 61, 49], [69, 66, 61, 49], [69, 66, 61, 47], [69, 66, 61, 47], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 59, 49], [68, 65, 59, 49], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [71, 66, 59, 56], [71, 66, 59, 56], [71, 65, 59, 56], [71, 65, 59, 56], [69, 66, 61, 57], [69, 66, 61, 57], [69, 66, 61, 59], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [69, 66, 61, 54], [69, 66, 61, 54], [69, 66, 61, 52], [69, 66, 

 ### Baseline

In [26]:
def baseline():
    def build_dataset_baseline(chorales):
        min_len = len(min(chorales, key=len))
        print(min_len)

        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def trunc(chorale):
            return chorale[:min_len+1]

        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        dataset = dataset.map(trunc)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
    #     for d in dataset:
    #         tf.print(d)
    #         break
        return dataset


    train_dataset = build_dataset_baseline(train_np_list)
    val_dataset = build_dataset_baseline(val_np_list)
    test_dataset = build_dataset_baseline(test_np_list)
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(1, input_shape=[None, 1])
    ])
    model.compile(loss='mse',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=[keras.metrics.mean_squared_error])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20, return_sequences=True),
        keras.layers.SimpleRNN(1),
    ])
    model.compile(loss='mse',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=[keras.metrics.mean_squared_error])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
    
baseline()

100
128
128
Epoch 1/7
8/8 [==============================] - 1s 31ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 2/7
8/8 [==============================] - 0s 16ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 3/7
8/8 [==============================] - 0s 15ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 4/7
8/8 [==============================] - 0s 14ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 5/7
8/8 [==============================] - 0s 15ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 6/7
8/8 [==============================] - 0s 14ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - va

2021-10-06 16:25:33,079 : WARNING : 5 out of the last 7 calls to <function Model.make_predict_function.<locals>.predict_function at 0x15816d040> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


### Sparse categorical entropy

In [126]:
import sys

def min_max_notes(all_dataset_chorales=()):
    min_note = None
    max_note = None
    def _min_max_notes():
        nonlocal min_note
        nonlocal max_note
        if min_note:
            return min_note, max_note
        min_note = sys.maxsize
        max_note = -sys.maxsize - 1
        for chorales in all_dataset_chorales:
            for ch in chorales:
                ch = np.array(ch)
                min_note = ch[ch > 0].min() if ch[ch > 0].min() < min_note else min_note
                max_note = ch.max() if ch.max() > max_note else max_note
            
        return min_note, max_note

    return _min_max_notes()


def predict_next_notes(model, initial_chorale, length=25, model_return_seq=False):
    next_notes = []
    seq = initial_chorale
    for n_note in range(length):
        prediction = model.predict(seq, batch_size=None)
        note = np.argmax(prediction, axis=1)
        if model_return_seq:
            note = note[0]
        next_notes.append(note[0]) # prediction is a list
        seq = np.append(seq, note.reshape((1, -1, 1)), axis=1)
    return next_notes


# replace the first note with a generated one
def predict_next_notes_with_shift(model, initial_chorale, length=25, model_return_seq=False):
    next_notes = []
    seq = initial_chorale
    for n_note in range(length):
        prediction = model.predict(seq, batch_size=None)
        note = np.argmax(prediction, axis=1)
        if model_return_seq:
            note = note[0]
        next_notes.append(note[0]) # prediction is a list
        seq = np.append(seq[:, 1:, :], note.reshape((1, -1, 1)), axis=1)
    return next_notes


min_note, max_note = min_max_notes((train_np_list, val_np_list, test_np_list))
num_notes = (max_note - min_note + 1) + 1 # different notes + 0 (no note) note
print(min_note, max_note)

36 81


In [67]:
def sparse_categorical_entropy():
    def build_dataset_sce(chorales):
        min_len = len(min(chorales, key=len))
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def trunc(chorale):
            return chorale[:min_len+1]
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        dataset = dataset.map(trunc)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_sce(train_np_list)
    val_dataset = build_dataset_sce(val_np_list)
    test_dataset = build_dataset_sce(test_np_list)
    
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20),
        keras.layers.Dense(num_notes, activation="softmax")
    ])
    for d in train_dataset:
        tf.print(model(d[0]).shape)
        break
    
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
        
    
sparse_categorical_entropy()

TensorShape([32, 47])
Epoch 1/50
8/8 [==============================] - 3s 56ms/step - loss: 3.2448 - accuracy: 0.1266 - val_loss: 2.8591 - val_accuracy: 0.1184
Epoch 2/50
8/8 [==============================] - 0s 27ms/step - loss: 2.6639 - accuracy: 0.1659 - val_loss: 2.7154 - val_accuracy: 0.0921
Epoch 3/50
8/8 [==============================] - 0s 27ms/step - loss: 2.5204 - accuracy: 0.1572 - val_loss: 2.6624 - val_accuracy: 0.0921
Epoch 4/50
8/8 [==============================] - 0s 27ms/step - loss: 2.4773 - accuracy: 0.1572 - val_loss: 2.7170 - val_accuracy: 0.0526
Epoch 5/50
8/8 [==============================] - 0s 28ms/step - loss: 2.4559 - accuracy: 0.1572 - val_loss: 2.6927 - val_accuracy: 0.0395
Epoch 6/50
8/8 [==============================] - 0s 26ms/step - loss: 2.4374 - accuracy: 0.1616 - val_loss: 2.6426 - val_accuracy: 0.0789
Epoch 7/50
8/8 [==============================] - 0s 27ms/step - loss: 2.4246 - accuracy: 0.1616 - val_loss: 2.6767 - val_accuracy: 0.0658
Epoch

2021-09-30 20:03:16,918 : WARNING : 6 out of the last 8 calls to <function Model.make_predict_function.<locals>.predict_function at 0x156472670> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


array([35, 35, 35, 35, 35, 35, 35,  8, 35, 35, 35, 35, 35, 35, 35, 38, 35,
       35, 35, 38, 38, 35, 35, 38, 35, 38, 38, 35, 35, 32, 35, 35])
array([35, 38, 35, 38, 35, 35, 35, 35, 35, 35, 35, 35, 35, 38, 35, 35, 38,
       35, 35, 38, 35, 35, 38, 35, 38, 38, 35, 35, 38, 35, 35, 35])
array([35, 35, 35, 38, 38, 38, 35, 35, 38, 35, 37, 38])


### Sparse categorical entropy. Larger datasets

In [81]:
def sparse_categorical_entropy_larger_dataset():
    def build_dataset_sceld(chorales):
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def win(chorale):
            window_size = 32
            window_shift = 16
            ds = tf.data.Dataset.from_tensor_slices(chorale).window(window_size, shift=window_shift, drop_remainder=True)
            # flatten windows created by window() function
            ds = ds.flat_map(lambda window: window.batch(window_size))
            return ds
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        # flatten dataset created in win()
        dataset = dataset.flat_map(win)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_sceld(train_np_list)
    val_dataset = build_dataset_sceld(val_np_list)
    test_dataset = build_dataset_sceld(test_np_list)
    
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20),
        keras.layers.Dense(num_notes, activation="softmax")
    ])
    for d in train_dataset:
        tf.print(model(d[0]).shape)
        break
    
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    # Shallow model doesn't need much training
    model.fit(train_dataset, epochs=3, validation_data=val_dataset)
    
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
        break
        
    
sparse_categorical_entropy_larger_dataset()

TensorShape([32, 47])
Epoch 1/3
425/425 [==============================] - 5s 8ms/step - loss: 2.7636 - accuracy: 0.1698 - val_loss: 2.4375 - val_accuracy: 0.2174
Epoch 2/3
425/425 [==============================] - 3s 8ms/step - loss: 2.4126 - accuracy: 0.2220 - val_loss: 2.2203 - val_accuracy: 0.3199
Epoch 3/3
425/425 [==============================] - 3s 8ms/step - loss: 2.1993 - accuracy: 0.2931 - val_loss: 1.9243 - val_accuracy: 0.3696
array([16, 16, 14, 16, 16,  9, 12, 12, 16, 16, 16, 16, 12, 14, 16, 16,  9,
        9,  9, 16, 16, 12, 16, 16, 16, 12,  9, 14,  9,  9,  9, 12])


### Sparse categorical entropy. LSTM

In [83]:
def sparse_categorical_entropy_lstm():
    def build_dataset_scelstm(chorales):
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def win(chorale):
            window_size = 32
            window_shift = 16
            ds = tf.data.Dataset.from_tensor_slices(chorale).window(window_size, shift=window_shift, drop_remainder=True)
            # flatten windows created by window() function
            ds = ds.flat_map(lambda window: window.batch(window_size))
            return ds
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        # flatten dataset created in win()
        dataset = dataset.flat_map(win)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_scelstm(train_np_list)
    val_dataset = build_dataset_scelstm(val_np_list)
    test_dataset = build_dataset_scelstm(test_np_list)
    
    n_embedding_dims = 5
    model = keras.models.Sequential([
        keras.layers.LSTM(20, return_sequences=True, input_shape=(None, 1)),
        keras.layers.LSTM(20, return_sequences=False),
        keras.layers.Dense(num_notes, activation="softmax")
    ])

    for d in train_dataset:
        tf.print(model(d[0]).shape)
        break
    
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    model.fit(train_dataset, epochs=7, validation_data=val_dataset)
    
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
        break
        
    
sparse_categorical_entropy_lstm()

TensorShape([32, 47])
Epoch 1/7
425/425 [==============================] - 9s 15ms/step - loss: 1.9485 - accuracy: 0.3809 - val_loss: 1.3181 - val_accuracy: 0.4940
Epoch 2/7
425/425 [==============================] - 6s 13ms/step - loss: 1.0256 - accuracy: 0.6529 - val_loss: 0.7154 - val_accuracy: 0.7826
Epoch 3/7
425/425 [==============================] - 6s 14ms/step - loss: 0.7096 - accuracy: 0.7765 - val_loss: 0.6507 - val_accuracy: 0.7578
Epoch 4/7
425/425 [==============================] - 6s 14ms/step - loss: 0.5825 - accuracy: 0.8142 - val_loss: 0.3752 - val_accuracy: 0.9107
Epoch 5/7
425/425 [==============================] - 6s 13ms/step - loss: 0.4877 - accuracy: 0.8498 - val_loss: 0.5630 - val_accuracy: 0.7459
Epoch 6/7
425/425 [==============================] - 6s 13ms/step - loss: 0.3989 - accuracy: 0.8786 - val_loss: 0.3935 - val_accuracy: 0.8875
Epoch 7/7
425/425 [==============================] - 6s 14ms/step - loss: 0.4187 - accuracy: 0.8786 - val_loss: 0.7115 - val_a

### Sparse categorical entropy. Deep network

In [110]:
def sparse_categorical_entropy_deep_network1():
    def build_dataset_scedn1(chorales):
        min_len = len(min(chorales, key=len))
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def win(chorale):
            window_size = 32
            window_shift = 16
            ds = tf.data.Dataset.from_tensor_slices(chorale).window(window_size, shift=window_shift, drop_remainder=True)
            # flatten windows created by window() function
            ds = ds.flat_map(lambda window: window.batch(window_size))
            return ds
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        # flatten dataset created in win()
        dataset = dataset.flat_map(win)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_scedn1(train_np_list)
    val_dataset = build_dataset_scedn1(val_np_list)
    test_dataset = build_dataset_scedn1(test_np_list)
    
    n_embedding_dims = 5
    model = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=(None, 1)),
        keras.layers.Conv1D(20, kernel_size=4, padding="valid", activation="relu", strides=2),
        keras.layers.GRU(20, return_sequences=True),
        keras.layers.GRU(20, return_sequences=False),
        keras.layers.Dense(num_notes, activation="softmax")
    ])

    for d in train_dataset:
        tf.print(model(d[0]).shape)
        break
    
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    model.fit(train_dataset, epochs=7, validation_data=val_dataset)
    
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
        break
        
    
sparse_categorical_entropy_deep_network1()

TensorShape([32, 47])
Epoch 1/7


KeyboardInterrupt: 

In [122]:
def sparse_categorical_entropy_deep_network2():
    def build_dataset_scedn2(chorales):
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def win(chorale):
            window_size = 32
            window_shift = 16
            ds = tf.data.Dataset.from_tensor_slices(chorale).window(window_size, shift=window_shift, drop_remainder=True)
            # flatten windows created by window() function
            ds = ds.flat_map(lambda window: window.batch(window_size))
            return ds
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        # flatten dataset created in win()
        dataset = dataset.flat_map(win)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_scedn2(train_np_list)
    val_dataset = build_dataset_scedn2(val_np_list)
    test_dataset = build_dataset_scedn2(test_np_list)
    
    n_embedding_dims = 5
    model = keras.models.Sequential([
        keras.layers.Embedding(input_dim=num_notes, output_dim=n_embedding_dims,
                               input_shape=[None]),
        keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
        keras.layers.BatchNormalization(),
        keras.layers.LSTM(256, return_sequences=False),
        keras.layers.Dense(num_notes, activation="softmax")
    ])

    for d in train_dataset:
        tf.print(model(d[0]).shape)
        break
    
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    model.fit(train_dataset, epochs=7, validation_data=val_dataset)
    
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
        break
        
    for d in test_dataset:
        initial_chorale = tf.reshape(d[0][0], [1, -1, 1]) # reshape to 1 batch
        predicted_next_notes = predict_next_notes(model, initial_chorale)
        print(f"intial chorale: {initial_chorale}")
        print(f"predicted_next_notes: {predicted_next_notes}")
        
        predicted_next_notes = predict_next_notes_with_shift(model, initial_chorale)
        print(f"intial chorale: {initial_chorale}")
        print(f"predicted_next_notes: {predicted_next_notes}")
        break
sparse_categorical_entropy_deep_network2()

TensorShape([32, 47])
Epoch 1/7
425/425 [==============================] - 19s 37ms/step - loss: 0.3634 - accuracy: 0.9071 - val_loss: 0.4648 - val_accuracy: 0.8604
Epoch 2/7
425/425 [==============================] - 16s 38ms/step - loss: 0.1101 - accuracy: 0.9743 - val_loss: 0.2741 - val_accuracy: 0.9337
Epoch 3/7
425/425 [==============================] - 16s 37ms/step - loss: 0.0709 - accuracy: 0.9854 - val_loss: 0.1438 - val_accuracy: 0.9609
Epoch 4/7
425/425 [==============================] - 15s 35ms/step - loss: 0.0859 - accuracy: 0.9789 - val_loss: 0.2828 - val_accuracy: 0.9304
Epoch 5/7
425/425 [==============================] - 15s 35ms/step - loss: 0.0732 - accuracy: 0.9839 - val_loss: 0.0471 - val_accuracy: 0.9912
Epoch 6/7
425/425 [==============================] - 15s 35ms/step - loss: 0.0754 - accuracy: 0.9846 - val_loss: 0.1976 - val_accuracy: 0.9512
Epoch 7/7
425/425 [==============================] - 15s 35ms/step - loss: 0.0661 - accuracy: 0.9857 - val_loss: 0.2118 

### Sparse categorical entropy. Deep network. Predicting sequence

In [ ]:
def sparse_categorical_entropy_deep_network2_seq():
    def build_dataset_scedn2s(chorales):
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def win(chorale):
            window_size = 32
            window_shift = 16
            ds = tf.data.Dataset.from_tensor_slices(chorale).window(window_size, shift=window_shift, drop_remainder=True)
            # flatten windows created by window() function
            ds = ds.flat_map(lambda window: window.batch(window_size))
            return ds
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[1:]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        # flatten dataset created in win()
        dataset = dataset.flat_map(win)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_scedn2s(train_np_list)
    val_dataset = build_dataset_scedn2s(val_np_list)
    test_dataset = build_dataset_scedn2s(test_np_list)
    
    n_embedding_dims = 5
    model = keras.models.Sequential([
        keras.layers.Embedding(input_dim=num_notes, output_dim=n_embedding_dims,
                               input_shape=[None]),
        keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
        keras.layers.BatchNormalization(),
        keras.layers.LSTM(256, return_sequences=True),
        keras.layers.Dense(num_notes, activation="softmax")
    ])

    for d in train_dataset:
        tf.print(model(d[0]).shape)
        break
    
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    model.fit(train_dataset, epochs=7, validation_data=val_dataset)
    
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
        break
        
    
sparse_categorical_entropy_deep_network2_seq()

TensorShape([32, 31, 47])
Epoch 1/7
